In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import json

# = = = = = = = = = = = = = = =

import re
import html as html_lib

# = = = = = = = = = = = = = = =

import itertools

In [2]:
directory_in = './file/2.url'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_dict_correct_2, list_series_error, df_input = [], [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                                                header=0,
                                                                                                dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：10



,No,Url
0,1,https://toyota.oempartsonline.com/oem-parts/to...
1,2,https://toyota.oempartsonline.com/oem-parts/to...
2,3,https://toyota.oempartsonline.com/oem-parts/to...
3,4,https://toyota.oempartsonline.com/oem-parts/to...
4,5,https://toyota.oempartsonline.com/oem-parts/to...
5,6,https://toyota.oempartsonline.com/oem-parts/to...
6,7,https://toyota.oempartsonline.com/oem-parts/to...
7,8,https://toyota.oempartsonline.com/oem-parts/to...
8,9,https://toyota.oempartsonline.com/oem-parts/to...
9,10,https://toyota.oempartsonline.com/oem-parts/to...


In [4]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                     1
 Url    https://toyota.oempartsonline.com/oem-parts/to...
 Name: 0, dtype: object)

In [5]:
import time

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request = crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            dict_ = json.loads(html.xpath('//script[@type="application/json"]/text()')[0])
            if dict_:
                with open('./part.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'in_rp_catalog': True,
 'source': 'product',
 'sku': '17805-0P050',
 'sku_stripped': '178050p050',
 'catalog': 'toyota',
 'price': 74.69,
 'msrp': 109.47,
 'images': [{'main': {'url': '//dz310nzuyimx0.cloudfront.net/strapr1/d3ee6b7275a0ad5c3a0ab8dc0b7d8bd9/a7ee74bed9bf1e534d4255d44631f547.png',
    'width': 640,
    'height': 471.09278350515467},
   'preview': {'url': '//dz310nzuyimx0.cloudfront.net/strapr1/d3ee6b7275a0ad5c3a0ab8dc0b7d8bd9/31071a2717268639a55188a717f8dbd1.png',
    'width': 300,
    'height': 220.82474226804126},
   'thumb': {'url': '//dz310nzuyimx0.cloudfront.net/strapr1/d3ee6b7275a0ad5c3a0ab8dc0b7d8bd9/93f63df316823116c87337cb900623ee.png',
    'width': 150,
    'height': 110.41237113402063},
   'type': 'PART',
   'image_id': 'a7ee74bed9bf1e534d4255d44631f547',
   'alt_text': '',
   'is_primary': False,
   'is_visible': True,
   'filename': '',
   'has_watermark': 'Unknown',
   'md5': '',
   'perceptual_hash': '',
   'product_sku_stripped': None,
   'hotspot': {'ref

In [6]:
dict_src, list_image = {}, dict_['images']
for i, image in enumerate(list_image):
    dict_src[str(i)] = f'''https:{image['main']['url'].strip()}''' if image['main']['url'].strip().startswith('//') else image['main']['url'].strip()

dict_src

{'0': 'https://dz310nzuyimx0.cloudfront.net/strapr1/d3ee6b7275a0ad5c3a0ab8dc0b7d8bd9/a7ee74bed9bf1e534d4255d44631f547.png',
 '1': 'https://dz310nzuyimx0.cloudfront.net/strapr1/d3ee6b7275a0ad5c3a0ab8dc0b7d8bd9/3f1e64d6f0207d22b573f76c7498f251.png',
 '2': 'https://dz310nzuyimx0.cloudfront.net/strapr1/d3ee6b7275a0ad5c3a0ab8dc0b7d8bd9/5c710b87f50152caf233e836a5c3f61e.png'}

In [7]:
list_vehicle, list_dict_fitment = [], dict_['fitment']
for dict_fitment in list_dict_fitment:
    dict_fitment.pop('idx')
    list_key = dict_fitment.keys()
    list_value = [dict_fitment[key] if type(dict_fitment[key]) == list else [dict_fitment[key]] for key in list_key]
    list_value.append([f'{dict_['catalog'].title().strip()};{dict_['sku'].strip()}'])
    list_key = list(list_key) + ['JOIN_CPN']

    product_combination = itertools.product(*list_value)

    for combination in product_combination:
        list_vehicle.append(dict(zip(list_key, combination)))

list_vehicle

[{'year': '2016',
  'make': 'Toyota',
  'model': 'Highlander',
  'trims': 'Hybrid Limited',
  'engines': '3.5L V6 - Electric/Gas',
  'JOIN_CPN': 'Toyota;17805-0P050'},
 {'year': '2015',
  'make': 'Toyota',
  'model': 'Highlander',
  'trims': 'Hybrid Limited',
  'engines': '3.5L V6 - Electric/Gas',
  'JOIN_CPN': 'Toyota;17805-0P050'},
 {'year': '2014',
  'make': 'Toyota',
  'model': 'Highlander',
  'trims': 'Hybrid Limited',
  'engines': '3.5L V6 - Electric/Gas',
  'JOIN_CPN': 'Toyota;17805-0P050'}]

In [8]:
subtitle = html.xpath('//div[@class="page-builder-layout-module desktop-only"]/descendant::p[@class="product-subtitle"]/text()')[0].strip() if html.xpath('//div[@class="page-builder-layout-module desktop-only"]/descendant::p[@class="product-subtitle"]/text()') else ''

subtitle

'2014-2016 Toyota Highlander - 178050p050'

In [9]:
list_li, dict_count = html.xpath('//h2[@class="product-details-title"]/following-sibling::ul[@class="field-list"][1]/li'), {}
for li in list_li:
    key = re.sub(r'<.*?>', '', etree.tostring(li.xpath('./span[@class="list-label"]')[0]).decode('utf-8'))
    key = re.sub(r' +', ' ', key.strip())
    key = html_lib.unescape(key)
    key = key.removesuffix(':').strip()
    if key == 'Brand':
        continue
    elif key in dict_count:
        dict_count[key] += 1
    else:
        dict_count[key] = 1

dict_detail, dict_no = {}, {}
for i, li in enumerate(list_li):
    key = re.sub(r'<.*?>', '', etree.tostring(li.xpath('./span[@class="list-label"]')[0]).decode('utf-8'))
    key = re.sub(r' +', ' ', key.strip())
    key = html_lib.unescape(key)
    key = key.removesuffix(':').strip()
    if key == 'Brand':
        list_src = li.xpath('./span[@class="list-value"]/div[@class="part-manufacturer"]/div[@class="manufacturer-logos"]/img/@src')
        list_src = ['.'.join(src.split('/')[-1].split('.')[:-1]).title().strip() for src in list_src]
        dict_detail[str(i)] = {key: ';'.join(list_src)}
    else:
        value = re.sub(r'<.*?>', '', etree.tostring(li.xpath('./span[contains(@class, "list-value")]')[0]).decode('utf-8'))
        value = re.sub(r' +', ' ', value.strip())
        value = html_lib.unescape(value)
        value = value.removesuffix(':').strip()
        if dict_count[key] == 1:
            dict_detail[str(i)] = {key: value}
        else:
            if key in dict_no:
                dict_no[key] += 1
            else:
                dict_no[key] = 1
            dict_detail[str(i)] = {f'{key}-{dict_no[key]}': value}

dict_detail

{'0': {'Brand': 'Toyota;Scion'},
 '1': {'SKU': '17805-0P050'},
 '2': {'Positions': 'Lower'},
 '3': {'Other Names': 'Resonator Sub Assembly'},
 '4': {'Description': '3.5L, with hybrid, 2014-16, lower.'},
 '5': {'Condition': 'New'}}

In [10]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Catalog': dict_['catalog'].title().strip(),
                          'Part Number': dict_['sku'].strip(),
                          'Title': dict_['title'].strip(),
                          'Subtitle': subtitle,
                          'Price': dict_['price'],
                          'Picture': '',
                          'Url': url_request,
                          'Json_Src': json.dumps(dict_src),
                          'Json_Detail': json.dumps(dict_detail),
                          'JOIN_CPN': f'{dict_['catalog'].title().strip()};{dict_['sku'].strip()}'})

list_dict_correct

[{'No': 1,
  'Catalog': 'Toyota',
  'Part Number': '17805-0P050',
  'Title': 'Resonator - Toyota (17805-0P050)',
  'Subtitle': '2014-2016 Toyota Highlander - 178050p050',
  'Price': 74.69,
  'Picture': '',
  'Url': 'https://toyota.oempartsonline.com/oem-parts/toyota-resonator-178050p050',
  'Json_Src': '{"0": "https://dz310nzuyimx0.cloudfront.net/strapr1/d3ee6b7275a0ad5c3a0ab8dc0b7d8bd9/a7ee74bed9bf1e534d4255d44631f547.png", "1": "https://dz310nzuyimx0.cloudfront.net/strapr1/d3ee6b7275a0ad5c3a0ab8dc0b7d8bd9/3f1e64d6f0207d22b573f76c7498f251.png", "2": "https://dz310nzuyimx0.cloudfront.net/strapr1/d3ee6b7275a0ad5c3a0ab8dc0b7d8bd9/5c710b87f50152caf233e836a5c3f61e.png"}',
  'Json_Detail': '{"0": {"Brand": "Toyota;Scion"}, "1": {"SKU": "17805-0P050"}, "2": {"Positions": "Lower"}, "3": {"Other Names": "Resonator Sub Assembly"}, "4": {"Description": "3.5L, with hybrid, 2014-16, lower."}, "5": {"Condition": "New"}}',
  'JOIN_CPN': 'Toyota;17805-0P050'}]

In [11]:
list_dict_correct_2.extend(list_vehicle)

list_dict_correct_2

[{'year': '2016',
  'make': 'Toyota',
  'model': 'Highlander',
  'trims': 'Hybrid Limited',
  'engines': '3.5L V6 - Electric/Gas',
  'JOIN_CPN': 'Toyota;17805-0P050'},
 {'year': '2015',
  'make': 'Toyota',
  'model': 'Highlander',
  'trims': 'Hybrid Limited',
  'engines': '3.5L V6 - Electric/Gas',
  'JOIN_CPN': 'Toyota;17805-0P050'},
 {'year': '2014',
  'make': 'Toyota',
  'model': 'Highlander',
  'trims': 'Hybrid Limited',
  'engines': '3.5L V6 - Electric/Gas',
  'JOIN_CPN': 'Toyota;17805-0P050'}]

In [12]:
crawler_status = True

crawler_status

True

In [13]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://toyota.oempartsonline.com/oem-parts/to...
 Request_Url    https://toyota.oempartsonline.com/oem-parts/to...
 Name: 0, dtype: object]

In [14]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No', 'Part Number'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct['No'] = range(1, len(df_correct) + 1)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct

,No,Catalog,Part Number,Title,Subtitle,Price,Picture,Url,Json_Src,Json_Detail,JOIN_CPN
0,1,Toyota,17805-0P050,Resonator - Toyota (17805-0P050),2014-2016 Toyota Highlander - 178050p050,74.69,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://dz310nzuyimx0.cloudfront.net/st...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;17805-0P050


In [15]:
if list_dict_correct_2:
    df_correct = pd.DataFrame(list_dict_correct_2)
    df_correct['year'] = df_correct['year'].astype(int)
    df_correct = df_correct.sort_values(by=['JOIN_CPN', 'make', 'model', 'year'],
                                        ascending=[True, True, True, False],
                                        ignore_index=True)
    df_correct.to_excel('./test_part-vehicle.xlsx', index=False)

df_correct

,year,make,model,trims,engines,JOIN_CPN
0,2016,Toyota,Highlander,Hybrid Limited,3.5L V6 - Electric/Gas,Toyota;17805-0P050
1,2015,Toyota,Highlander,Hybrid Limited,3.5L V6 - Electric/Gas,Toyota;17805-0P050
2,2014,Toyota,Highlander,Hybrid Limited,3.5L V6 - Electric/Gas,Toyota;17805-0P050


In [16]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)

df_error

,No,Url,Request_Url
0,1,https://toyota.oempartsonline.com/oem-parts/to...,https://toyota.oempartsonline.com/oem-parts/to...
